In [1]:
import os
from os.path import *

In [2]:
validation='/media/Gui2/thermix/ARTraining/trained_models/thermix_38a/thermix_38a_testing.txt'
'''
if !predictions, run:
    go
    cd ccv/bin
    ./cnnclassify /media/Gui2/thermix/ARTraining/trained_models/thermix_38a/thermix_38a_testing.txt \
                    /media/Gui2/thermix/ARTraining/trained_models/thermix_38a/thermix_38a.sqlite3 \
                    /media/Gui2/thermix/dataset_poses_people_background \
                    | tee /media/Gui2/thermix/ARTraining/trained_models/thermix_38a/thermix_38a_testing_predictions.txt
'''
predictions='/media/Gui2/thermix/ARTraining/trained_models/thermix_38a/thermix_38a_testing_predictions.txt'

In [3]:
def get_validation_vs_prediction():
    
    val_file = open(validation, 'r')
    val_lines = val_file.readlines()
    val_file.close()

    pred_file = open(predictions, 'r')
    pred_lines = pred_file.readlines()
    pred_file.close()
        
    vals = []
    for line in val_lines:
        c, p = line.rstrip().split(' ')
        vals.append([c, p])
        
    preds = []
    for line in pred_lines:
        p, c, s = line.rstrip().split(' ')
        preds.append([c, p, s])
        
    assert len(vals) == len(preds)
    
    all_data = []
    for v in vals:
        for p in preds:
            if v[1] in p:
                all_data.append([v[1], v[0], p[0], p[2]])
    
    # all_data[i] = [<path_i>, <real_class_i>, <predicted_class_i>, <prediction_score_i>]
    return all_data

In [56]:
def confusion_matrix(population):
    print 'positive => lying, sitting, standing, people (interesting)'
    print 'negative => background (not interesting)'
    print '-----'
    
    tp = []
    tn = []
    fp = []
    fn = []

    condition_positive = [[y,h] for [y,h] in data if is_positive(y)]
    condition_negative = [[y,h] for [y,h] in data if is_negative(y)]
    
    assert(len(population) == len(condition_negative)+len(condition_positive))
    
    for y,h in population:
        # true positive => y == 1,2,3,4 and h == 1,2,3,4
        if is_positive(y) and is_positive(h):
            tp.append([y,h])
        # get true negatives
        elif is_negative(y) and is_negative(h):
            tn.append([y,h])
        # get false positives => y == 5 and h == 1,2,3,4
        elif is_negative(y) and is_positive(h):
            fp.append([y,h])
        # get false negatives
        elif is_positive(y) and is_negative(h):
            fn.append([y,h])
            
    assert(len(tp)+len(tn)+len(fp)+len(fn) == len(population))
    
    sensitivity = count(tp)/count(condition_positive)
    specificity = count(tn)/count(condition_negative)
    precision = count(tp)/(count(tp)+count(fp))
    negative_predictive_value = count(tn)/(count(tn)+count(fn))
    accuracy = (count(tp)+count(tn))/(count(population))
    
    print 'accuracy: '+str(accuracy*100)+' %'
    print 'sensitivity (recall, true positive rate): '+str(sensitivity*100)+' %'
    print 'specificity (true negative rate): '+str(specificity*100)+' %'
    print 'precision (positive predictive value) : '+str(precision*100)+' %'
    print 'negative predictive value: '+str(negative_predictive_value*100)+' %'
    print 'fall-out (false positive rate): '+str((1-specificity)*100)+' %'
    print 'miss-rate (false negative rate): '+str((1-sensitivity)*100)+' %'
    print 'false-discovery-rate: '+str((1-precision)*100)+' %'
    
    
def is_positive(x):
    return (x == 1 or x == 2 or x == 3 or x == 4)

def is_negative(x):
    return (x == 5)

def count(arr):
    return float(len(arr))

In [61]:
print 'validation: '+os.path.basename(validation)+' ,predictions: '+os.path.basename(predictions)
print '-----'
data = [[int(y),int(h)] for _,y,h,_ in get_validation_vs_prediction()]
confusion_matrix(data)

validation: thermix_38a_testing.txt ,predictions: thermix_38a_testing_predictions.txt
-----
positive => lying, sitting, standing, people (interesting)
negative => background (not interesting)
-----
accuracy: 96.1865569273 %
sensitivity (recall, true positive rate): 96.5235173824 %
specificity (true negative rate): 94.7960618847 %
precision (positive predictive value) : 98.710352039 %
negative predictive value: 86.8556701031 %
fall-out (false positive rate): 5.20393811533 %
miss-rate (false-negative-rate): 3.47648261759 %
false-discovery-rate: 1.28964796096 %
